# Dipping the toe

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET

from imageio import imread
from skimage.transform import resize

%matplotlib inline

In [2]:
TRAIN_DIR = 'images/training_data/'
TEST_DIR = 'images/test_data_images/'

train_images = os.listdir(TRAIN_DIR + 'images/')
train_xml_files = os.listdir(TRAIN_DIR + 'labels/')

test_images = os.listdir(TEST_DIR + 'images/')

In [3]:
class_labels = ['Car', 'Pool']

## Creating a dataframe of all the images and  bounding boxes

In [13]:
def parse_xml(xml_file_path):
    root = ET.parse(xml_file_path).getroot()
    file_name = root.find('./filename').text
    
    records = []
    for obj in root.findall('./object'):
        bnb_box = obj.find('./bndbox')
        record = (
                    file_name,
                    int(obj.find('./name').text),   # Class label
                    float(bnb_box.find('xmin').text),
                    float(bnb_box.find('ymin').text),
                    float(bnb_box.find('xmax').text),
                    float(bnb_box.find('ymax').text)
                 )
        records.append(record)
    return file_name, records

X = []
X_boxes = []
for xml_name in train_xml_files:
    xml_file_path = os.path.join(TRAIN_DIR, 'labels/', xml_name)
    img_name, records = parse_xml(xml_file_path)
    
    img_file = os.path.join(TRAIN_DIR, 'images/', img_name)
    X.append([img_name, img_file, xml_name, xml_file_path])
    X_boxes.extend(records)
    

X_df = pd.DataFrame(X, columns=['img_name', 'img_path', 'xml_name', 'xml_path']).sort_values('img_path')
X_df.index = range(X_df.shape[0])
print(X_df.head())

X_boxes_df = pd.DataFrame(X_boxes, columns=['img_name', 'class_name', 'xmin', 'ymin', 'xmax', 'ymax']).sort_values('img_name')
X_boxes_df.index = range(X_boxes_df.shape[0])
print(X_boxes_df.head())

        img_name                                 img_path       xml_name  \
0  000000000.jpg  data/training_data/images/000000000.jpg  000000000.xml   
1  000000001.jpg  data/training_data/images/000000001.jpg  000000001.xml   
2  000000002.jpg  data/training_data/images/000000002.jpg  000000002.xml   
3  000000003.jpg  data/training_data/images/000000003.jpg  000000003.xml   
4  000000004.jpg  data/training_data/images/000000004.jpg  000000004.xml   

                                  xml_path  
0  data/training_data/labels/000000000.xml  
1  data/training_data/labels/000000001.xml  
2  data/training_data/labels/000000002.xml  
3  data/training_data/labels/000000003.xml  
4  data/training_data/labels/000000004.xml  
        img_name  class_name   xmin    ymin   xmax    ymax
0  000000000.jpg           1  58.47  152.31  69.58  163.43
1  000000000.jpg           1  10.32  205.68  21.43  216.80
2  000000000.jpg           1  40.37    0.00  50.97    3.89
3  000000000.jpg           1  46.67  

## Count: Cars vs Pools
### TODO

In [49]:
X_boxes_df.class_name.value_counts()

1    13022
2     3150
Name: class_name, dtype: int64

## Train test split

In [41]:
from sklearn.model_selection import train_test_split

X_train, X_validation, _, _ = train_test_split(X_df, X_df, test_size=0.15, random_state=2019)

X_train['img_dest'] = 'images/train/' + X_train.img_name
X_validation['img_dest'] = 'images/validation/' + X_validation.img_name

X_train['xml_dest'] = 'images/train/' + X_train.xml_name
X_validation['xml_dest'] = 'images/validation/' + X_validation.xml_name


/home/obed/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/obed/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [42]:
# Copy the images to train and validation directories
from shutil import copyfile

def copy_images_xml(df):
    for row in df.iterrows():
        row = row[1]
        copyfile(row.img_path, row.img_dest)
        copyfile(row.xml_path, row.xml_dest)

In [44]:
copy_images_xml(X_train)
copy_images_xml(X_validation)

In [55]:
X_train_bdf = X_boxes_df[X_boxes_df.img_name.isin(X_train.img_name)]
X_validation_bdf = X_boxes_df[X_boxes_df.img_name.isin(X_validation.img_name)]

print(X_train_bdf.img_name.unique().shape)
X_train_bdf.sample(5)

(2623,)


,img_name,class_name,xmin,ymin,xmax,ymax
5273,000001636.jpg,1,0.00,180.87,7.68,191.98
5243,000001621.jpg,1,14.28,78.21,25.39,89.33
4959,000001514.jpg,2,214.55,0.00,224.00,30.14
5573,000001740.jpg,1,209.22,61.42,220.33,72.53
9635,000003094.jpg,2,198.14,4.85,224.00,49.29


In [20]:
X_train_bdf.to_csv(TRAIN_DIR + 'train_labels.csv', index=False)
X_validation_bdf.to_csv(TRAIN_DIR + 'validation_labels.csv', index=False)

In [58]:
import pickle

with open('df.pickle', 'wb') as f:
    pickle.dump((X_train, X_train_bdf, X_validation, X_validation_bdf), f)

In [5]:
import pickle

with open('df.pickle', 'rb') as f:
    X_train, X_train_bdf, X_validation, X_validation_bdf = pickle.load(f)

## Converting the images to TFRecords

In [6]:
from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict

In [7]:
data = namedtuple('data', ['img_name', 'object'])
data

__main__.data

In [8]:
def split(df, group):
    data = namedtuple('data', ['img_name', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]

In [9]:
grouped = split(X_train_bdf, 'img_name')

In [17]:
grouped[0]

data(img_name='000000000.jpg', object=        img_name  class_name   xmin    ymin   xmax    ymax
0  000000000.jpg           1  46.67   41.29  57.79   52.40
1  000000000.jpg           1  58.47  152.31  69.58  163.43
2  000000000.jpg           1  10.32  205.68  21.43  216.80
3  000000000.jpg           1  40.37    0.00  50.97    3.89)

In [13]:
import tensorflow as tf
import io
from PIL import Image

def create_tf_example(group, path):
    with tf.gfile.GFile(os.path.join(path, '{}'.format(group.img_name)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    
    #image = load_image(os.path.join(path, group.img_name))
    
    width, height = image.size

    img_name = group.img_name.encode('utf8')
    image_format = b'jpg'
    # check if the image format is matching with your images.
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(class_labels[row['class_name'] - 1].encode('utf8'))
        classes.append(row['class_name'])
        

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/img_name': dataset_util.bytes_feature(img_name),
        'image/source_id': dataset_util.bytes_feature(img_name),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

create_tf_example(grouped[0],'../Tensorflow/workspace/esri/benchmark/images/train/')

features {
  feature {
    key: "image/encoded"
    value {
      bytes_list {
        value: "\377\330\377\340\000\020JFIF\000\001\001\000\000\001\000\001\000\000\377\333\000C\000\005\003\004\004\004\003\005\004\004\004\005\005\005\006\007\014\010\007\007\007\007\017\013\013\t\014\021\017\022\022\021\017\021\021\023\026\034\027\023\024\032\025\021\021\030!\030\032\035\035\037\037\037\023\027\"$\"\036$\034\036\037\036\377\333\000C\001\005\005\005\007\006\007\016\010\010\016\036\024\021\024\036\036\036\036\036\036\036\036\036\036\036\036\036\036\036\036\036\036\036\036\036\036\036\036\036\036\036\036\036\036\036\036\036\036\036\036\036\036\036\036\036\036\036\036\036\036\036\036\036\036\377\300\000\021\010\000\340\000\340\003\001\"\000\002\021\001\003\021\001\377\304\000\035\000\000\002\002\003\001\001\001\000\000\000\000\000\000\000\000\000\005\006\004\007\002\003\010\001\000\t\377\304\000@\020\000\002\001\002\005\002\003\006\003\006\004\006\002\003\000\000\001\002\003\004\021\000\005\

In [11]:
def write_tf_records(df, image_path, tfrecord_path):
    writer = tf.python_io.TFRecordWriter(tfrecord_path)
    grouped = split(df, 'img_name')

    for group in grouped:
        tf_example = create_tf_example(group, image_path)
        writer.write(tf_example.SerializeToString())
    writer.close()

In [16]:
write_tf_records(X_train_bdf, 'images/train/', 'images/train.record')
write_tf_records(X_validation_bdf, 'images/validation/', 'images/validation.record')